In [1]:
import pandas as pd
import json
import numpy
import json
from collections import defaultdict, Counter
from tqdm import tqdm
import random
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import time

# Dataset for Reward Model

In [2]:
with open(
    "../data/base/interactions_v3.json", encoding="cp437", errors="ignore"
) as json_file:
    data = json.load(json_file)

dataframe = pd.DataFrame(data)
dataframe.loc[8705]["interaction"][1]["role"] = "assistant"
dataframe.head()

,confidence,interaction,sol_id,interaction_id,label,chat,length,number_split
0,2,"[{'role': 'system', 'content': 'Answer in engl...",1000851,4747078,0,|SYS| Answer in english. Show your thinking st...,955,1
1,4,"[{'role': 'system', 'content': 'You are a phys...",1000851,4390722,1,|SYS| You are a physics teacher. You always ch...,891,1
2,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4967894,1,|SYS| QCM\n\n|USER| Pour encoder la séquence d...,172,1
3,5,"[{'role': 'user', 'content': 'Pour encoder la ...",1001199,4944321,1,|USER| Pour encoder la séquence de lettres ABC...,248,0
4,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4039807,1,|SYS| QCM\n\n|USER| Pour encoder la séquence d...,207,1


In [3]:
with open("../data/base/solutions_v1.json") as fh:
    solutions = json.load(fh)
solutions_df = pd.DataFrame(solutions)
solutions_df.set_index("sol_id", inplace=True, drop=False)

In [4]:
def create_Chat(chat):
    interactions = chat[f"interaction"]
    sol_id = chat["sol_id"]

    question = ""
    try:
        solution = solutions_df.loc[sol_id]
        question += f"{solution['question']}\n"

        if solution["choices"] != None and not type(solution["choices"]) == float:
            question += f"Choices: {solution['choices']}\n"

    except:
        question = ""
        print("solution not found")
    new = "user: " + question + "\n" + "assistant: "
    wenlo = False
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            new += interaction["content"]
            wenlo = True
    if not wenlo:
        print("TABUU", chat["interaction_id"], sol_id)
    new += "\n\n"
    return new


dataframe["chat"] = dataframe.apply(create_Chat, axis=1)

In [5]:
def create_Chat(chat):
    interactions = chat[f"interaction"]
    count = 0
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            count += 1
    return count


dataframe["assistant_chat"] = dataframe.apply(create_Chat, axis=1)

In [6]:
with open(
    "../data/paraphrase_one_chat.json", encoding="cp437", errors="ignore"
) as json_file:
    one = json.load(json_file)
with open(
    "../data/paraphrase_many__chats.json", encoding="cp437", errors="ignore"
) as json_file:
    many = json.load(json_file)

paraphrase = pd.DataFrame(one + many)
paraphrase["assistant_chat"] = paraphrase["assistant_chat"].apply(lambda x: 1)
paraphrase["label"] = paraphrase["confidence"].apply(lambda x: 1)
print(len(paraphrase))
paraphrase.head()


def add_Assitant(example):
    parts = example["chat"].split("\n\n")
    parts[1] = "assistant: " + parts[1]
    final = "\n\n".join(parts)
    return final


paraphrase["chat"] = paraphrase.apply(add_Assitant, axis=1)
paraphrase.to_json("../data/paraphrase.json", orient="records", indent=4)

7080


In [7]:
dataframe = pd.concat([dataframe, paraphrase], ignore_index=True)
dataframe

,confidence,interaction,sol_id,interaction_id,label,chat,length,number_split,assistant_chat
0,2,"[{'role': 'system', 'content': 'Answer in engl...",1000851,4747078,0,user: Une conquille sphérique de rayon $R_1$ e...,955,1.0,2
1,4,"[{'role': 'system', 'content': 'You are a phys...",1000851,4390722,1,user: Une conquille sphérique de rayon $R_1$ e...,891,1.0,3
2,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4967894,1,user: Pour encoder la séquence de lettres ABCC...,172,1.0,2
3,5,"[{'role': 'user', 'content': 'Pour encoder la ...",1001199,4944321,1,user: Pour encoder la séquence de lettres ABCC...,248,0.0,2
4,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4039807,1,user: Pour encoder la séquence de lettres ABCC...,207,1.0,2
...,...,...,...,...,...,...,...,...,...
17285,4,"[{'role': 'system', 'content': 'Answer in engl...",2996157,856763,1,"user: Par définition, l'inductance propre ""L"" ...",152,NaN,1
17286,4,"[{'role': 'system', 'content': 'Select the cor...",2998595,856766,1,user: A muscle is controlled by a group of mot...,234,NaN,1
17287,4,"[{'role': 'system', 'content': 'Answer in engl...",2999488,856769,1,user: On prend $1 m^3$ d'air à l'extérieur en ...,258,NaN,1
17288,4,"[{'role': 'system', 'content': 'Vous allez aid...",2999633,856773,1,user: Parmi les séquences de lettres suivantes...,126,NaN,1


In [8]:
dataframe["length"] = dataframe["chat"].apply(lambda x: len(word_tokenize(x)))

In [9]:
dataframe[dataframe["length"] <= 512]

,confidence,interaction,sol_id,interaction_id,label,chat,length,number_split,assistant_chat
2,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4967894,1,user: Pour encoder la séquence de lettres ABCC...,234,1.0,2
3,5,"[{'role': 'user', 'content': 'Pour encoder la ...",1001199,4944321,1,user: Pour encoder la séquence de lettres ABCC...,233,0.0,2
4,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4039807,1,user: Pour encoder la séquence de lettres ABCC...,287,1.0,2
5,4,"[{'role': 'user', 'content': 'Assume that we h...",1001417,4906222,1,user: Assume that we have a convolutional neur...,447,0.0,1
6,5,"[{'role': 'system', 'content': 'You are a teac...",1002278,4150255,1,user: Which of the following statements is \te...,363,1.0,1
...,...,...,...,...,...,...,...,...,...
17285,4,"[{'role': 'system', 'content': 'Answer in engl...",2996157,856763,1,"user: Par définition, l'inductance propre ""L"" ...",154,NaN,1
17286,4,"[{'role': 'system', 'content': 'Select the cor...",2998595,856766,1,user: A muscle is controlled by a group of mot...,236,NaN,1
17287,4,"[{'role': 'system', 'content': 'Answer in engl...",2999488,856769,1,user: On prend $1 m^3$ d'air à l'extérieur en ...,260,NaN,1
17288,4,"[{'role': 'system', 'content': 'Vous allez aid...",2999633,856773,1,user: Parmi les séquences de lettres suivantes...,128,NaN,1


In [10]:
dataframe = dataframe[dataframe["length"] <= 512]
dataframe[["chat", "confidence"]].to_json(
    "../data/eval_dataset.json", orient="records", indent=4
)

In [11]:
def creating(example):
    id = example["sol_id"]
    interactions = dataframe[dataframe["sol_id"] == id]
    for i in range(len(interactions)):
        for j in range(len(interactions)):
            if i != j:
                global final
                first = interactions.iloc[i]
                second = interactions.iloc[j]
                if first.confidence < second.confidence:
                    final.append(
                        {
                            "final_chat_0": first.chat,
                            "final_chat_1": second.chat,
                            "label_0": first.confidence,
                            "label_1": second.confidence,
                            "length_0": first.length,
                            "length_1": second.length,
                            "sol_id": id,
                        }
                    )


final = []
solutions_df.apply(creating, axis=1)
len(final)

12011

In [12]:
final_df = pd.DataFrame(final)
final_df["label_1"].value_counts()

label_1
5    9440
4    2382
3     140
2      33
1      16
Name: count, dtype: int64

In [13]:
with open(
    "../data/base/ChatGPT_augmented.json", encoding="cp437", errors="ignore"
) as json_file:
    data = json.load(json_file)
positives = pd.DataFrame(data)
positives_new = positives[
    [
        "confidence_0",
        "confidence_1",
        "interaction_0",
        "interaction_1",
        "length_0",
        "length_1",
        "sol_id",
    ]
]
positives_new = positives_new.rename(
    columns={"confidence_0": "label_0", "confidence_1": "label_1"}
)


def create_Chat(chat):
    interactions = chat[f"interaction_{variable}"]
    sol_id = chat["sol_id"]

    question = ""
    try:
        solution = solutions_df.loc[sol_id]
        question += f"{solution['question']}\n"

        if solution["choices"] != None and not type(solution["choices"]) == float:
            question += f"Choices: {solution['choices']}\n"

    except:
        question = ""
        print("solution not found")
    new = "user: " + question + "\n" + "assistant: "
    wenlo = False
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            new += interaction["content"]
            wenlo = True
    if not wenlo:
        print("TABUUU")
    new += "\n\n"
    return new


for variable in range(2):
    positives_new[f"final_chat_{variable}"] = positives_new.apply(create_Chat, axis=1)


positives_new = positives_new.drop(["interaction_0", "interaction_1"], axis=1)
positives_new.to_json("wenlo.json", orient="records", indent=4)
with open("wenlo.json", encoding="cp437", errors="ignore") as json_file:
    data_positive = json.load(json_file)
!rm wenlo.json

In [14]:
positives_new

,label_0,label_1,length_0,length_1,sol_id,final_chat_0,final_chat_1
0,1,5,275,248,1001199,user: Pour encoder la séquence de lettres ABCC...,user: Pour encoder la séquence de lettres ABCC...
1,1,4,436,438,1001417,user: Assume that we have a convolutional neur...,user: Assume that we have a convolutional neur...
2,1,5,114,236,1002278,user: Which of the following statements is \te...,user: Which of the following statements is \te...
3,1,5,886,886,1002413,user: Consider the Gaussian mixture model wher...,user: Consider the Gaussian mixture model wher...
4,1,5,209,1105,1004520,user: Assume [mathjaxinline] a_0=a_2 [/mathjax...,user: Assume [mathjaxinline] a_0=a_2 [/mathjax...
...,...,...,...,...,...,...,...
2143,1,4,444,795,2995485,user: Consider a binary classification problem...,user: Consider a binary classification problem...
2144,1,5,161,285,2996487,user: Calculer le volume molaire d'un gaz idéa...,user: Calculer le volume molaire d'un gaz idéa...
2145,1,5,57,60,2997321,user: Integral under peak of the Fourier trans...,user: Integral under peak of the Fourier trans...
2146,1,5,97,81,2998144,user: Match one of the tools (a)-(e) to the fo...,user: Match one of the tools (a)-(e) to the fo...


In [22]:
final_list = final + data_positive
final_df = pd.DataFrame(final_list)
print(len(final_df))

14159


In [24]:
final_df["assistant_length_0"] = final_df["final_chat_0"].apply(
    lambda x: len(word_tokenize(x.split("assistant: ")[1]))
)
final_df["assistant_length_1"] = final_df["final_chat_1"].apply(
    lambda x: len(word_tokenize(x.split("assistant: ")[1]))
)
final_df = final_df[final_df["assistant_length_1"] > 0]

In [25]:
final_df = final_df[(final_df['label_1']-final_df['label_0']>=2)]
final_df = final_df[(final_df["length_0"] <= 512) & (final_df["length_1"] <= 512)]

In [26]:
# drop columns
final_df = final_df.drop(
    ["assistant_length_0", "assistant_length_1", "label_0", "label_1", "sol_id"], axis=1
)

In [27]:
final_df.to_json("../data/training_dataset.json", orient="records", indent=4)

# Dataset for RLHF

In [242]:
with open(
    "../data/base/interactions_v3.json", encoding="cp437", errors="ignore"
) as json_file:
    data = json.load(json_file)

dataframe = pd.DataFrame(data)
dataframe.loc[8705]["interaction"][1]["role"] = "assistant"
dataframe = dataframe[dataframe["confidence"] > 3]
dataframe.head()

,confidence,interaction,sol_id,interaction_id,label,chat,length,number_split
1,4,"[{'role': 'system', 'content': 'You are a phys...",1000851,4390722,1,|SYS| You are a physics teacher. You always ch...,891,1
2,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4967894,1,|SYS| QCM\n\n|USER| Pour encoder la séquence d...,172,1
3,5,"[{'role': 'user', 'content': 'Pour encoder la ...",1001199,4944321,1,|USER| Pour encoder la séquence de lettres ABC...,248,0
4,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4039807,1,|SYS| QCM\n\n|USER| Pour encoder la séquence d...,207,1
5,4,"[{'role': 'user', 'content': 'Assume that we h...",1001417,4906222,1,|USER| Assume that we have a convolutional neu...,438,0


In [243]:
def create_Chat(chat):
    interactions = chat[f"interaction"]
    sol_id = chat["sol_id"]

    question = ""
    try:
        solution = solutions_df.loc[sol_id]
        question += f"{solution['question']}\n"

        if solution["choices"] != None and not type(solution["choices"]) == float:
            question += f"Choices: {solution['choices']}\n"

    except:
        question = ""
        print("solution not found")
    new = "user: " + question + "\n" + "assistant: "
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            new += interaction["content"]

    new += "\n\n"
    return new


dataframe["chat"] = dataframe.apply(create_Chat, axis=1)
dataframe["length"] = dataframe["chat"].apply(lambda x: len(word_tokenize(x)))
dataframe

,confidence,interaction,sol_id,interaction_id,label,chat,length,number_split
1,4,"[{'role': 'system', 'content': 'You are a phys...",1000851,4390722,1,user: Une conquille sphérique de rayon $R_1$ e...,775,1
2,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4967894,1,user: Pour encoder la séquence de lettres ABCC...,234,1
3,5,"[{'role': 'user', 'content': 'Pour encoder la ...",1001199,4944321,1,user: Pour encoder la séquence de lettres ABCC...,233,0
4,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4039807,1,user: Pour encoder la séquence de lettres ABCC...,287,1
5,4,"[{'role': 'user', 'content': 'Assume that we h...",1001417,4906222,1,user: Assume that we have a convolutional neur...,447,0
...,...,...,...,...,...,...,...,...
10204,4,"[{'role': 'system', 'content': 'Vous allez aid...",2999633,4405298,1,user: Parmi les séquences de lettres suivantes...,469,1
10206,5,"[{'role': 'system', 'content': 'Act as a compu...",2999633,4198754,1,user: Parmi les séquences de lettres suivantes...,842,1
10207,5,"[{'role': 'system', 'content': 'Could you plea...",2999828,4834259,1,user: What does an ontology provide?\nChoices:...,192,1
10208,4,"[{'role': 'system', 'content': 'From now on, y...",2999828,4823002,1,user: What does an ontology provide?\nChoices:...,160,1


In [244]:
def create_Chat(chat):
    interactions = chat[f"interaction"]
    count = 0
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            count += 1
    return count


dataframe["assistant_chat"] = dataframe.apply(create_Chat, axis=1)

In [257]:
dataframe["length_assitant"] = dataframe["chat"].apply(
    lambda x: len(word_tokenize(x.split("assistant: ")[1]))
)
dataframe

,confidence,interaction,sol_id,interaction_id,label,chat,length,number_split,assistant_chat,length_assitant
1,4,"[{'role': 'system', 'content': 'You are a phys...",1000851,4390722,1,user: Une conquille sphérique de rayon $R_1$ e...,775,1,3,686
2,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4967894,1,user: Pour encoder la séquence de lettres ABCC...,234,1,2,108
3,5,"[{'role': 'user', 'content': 'Pour encoder la ...",1001199,4944321,1,user: Pour encoder la séquence de lettres ABCC...,233,0,2,107
4,4,"[{'role': 'system', 'content': 'QCM'}, {'role'...",1001199,4039807,1,user: Pour encoder la séquence de lettres ABCC...,287,1,2,161
5,4,"[{'role': 'user', 'content': 'Assume that we h...",1001417,4906222,1,user: Assume that we have a convolutional neur...,447,0,1,303
...,...,...,...,...,...,...,...,...,...,...
10204,4,"[{'role': 'system', 'content': 'Vous allez aid...",2999633,4405298,1,user: Parmi les séquences de lettres suivantes...,469,1,2,436
10206,5,"[{'role': 'system', 'content': 'Act as a compu...",2999633,4198754,1,user: Parmi les séquences de lettres suivantes...,842,1,2,809
10207,5,"[{'role': 'system', 'content': 'Could you plea...",2999828,4834259,1,user: What does an ontology provide?\nChoices:...,192,1,1,141
10208,4,"[{'role': 'system', 'content': 'From now on, y...",2999828,4823002,1,user: What does an ontology provide?\nChoices:...,160,1,1,109


In [260]:
chats = dataframe[["length", "chat", "assistant_chat"]]

In [261]:
chats.to_json("../data/chats.json", orient="records", indent=4)

In [262]:
chats[chats["assistant_chat"] == 1].to_json(
    "../data/chats_one_assistant.json", orient="records", indent=4
)

# Final augmentation

In [226]:
import gpt_wrapper
from gpt_wrapper.chat import Chat

gpt_wrapper.api_key = "ac845d8b-b72f-41f3-9392-df91cb57921a"
print(
    f"Budget: {(Chat.budget()['limit']-Chat.budget()['usage'])/(1000 * 1000)} Millions tokens"
)

Budget: 3.912894 Millions tokens


In [3]:
with open(
    "../data/base/interactions_v3.json", encoding="cp437", errors="ignore"
) as json_file:
    data = json.load(json_file)

dataframe = pd.DataFrame(data)
print(len(dataframe))

good = dataframe[dataframe["confidence"] > 3]
len(good)

10210


7091

In [4]:
def create_Chat(chat):
    interactions = chat[f"interaction"]
    count = 0
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            count += 1
    return count


good["assistant_chat"] = good.apply(create_Chat, axis=1)
good = good[
    ["confidence", "interaction", "interaction_id", "sol_id", "chat", "assistant_chat"]
]
good = good[good["assistant_chat"] == 1]
good

/var/folders/z2/hzn8xqy537x0xvfzqzp8jr580000gn/T/ipykernel_25004/2217401551.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good["assistant_chat"] = good.apply(create_Chat, axis=1)


,confidence,interaction,interaction_id,sol_id,chat,assistant_chat
5,4,"[{'role': 'user', 'content': 'Assume that we h...",4906222,1001417,|USER| Assume that we have a convolutional neu...,1
6,5,"[{'role': 'system', 'content': 'You are a teac...",4150255,1002278,|SYS| You are a teaching assistant answering q...,1
7,5,"[{'role': 'system', 'content': 'You are an edu...",4373321,1002278,|SYS| You are an educational assistant that an...,1
10,5,"[{'role': 'system', 'content': 'Tu es un profe...",4998890,1003546,|SYS| Tu es un professeur de physique. Réponds...,1
13,4,"[{'role': 'system', 'content': 'You are an edu...",4548615,1004520,|SYS| You are an educational assistant that an...,1
...,...,...,...,...,...,...
10196,5,"[{'role': 'user', 'content': ' Match one of th...",4384945,2998144,|USER| Match one of the tools (a)-(e) to the ...,1
10197,5,"[{'role': 'system', 'content': 'You are trying...",4994466,2998144,|SYS| You are trying to help students in quest...,1
10207,5,"[{'role': 'system', 'content': 'Could you plea...",4834259,2999828,|SYS| Could you please answer the multiple cho...,1
10208,4,"[{'role': 'system', 'content': 'From now on, y...",4823002,2999828,"|SYS| From now on, you will act as a computati...",1


In [6]:
with open("../data/base/solutions_v1.json") as fh:
    solutions = json.load(fh)
solutions_df = pd.DataFrame(solutions)
solutions_df.set_index("sol_id", inplace=True, drop=False)

In [7]:
def create_Chat(chat):
    interactions = chat[f"interaction"]
    sol_id = chat["sol_id"]

    question = ""
    try:
        solution = solutions_df.loc[sol_id]
        question += f"{solution['question']}\n"

        if solution["choices"] != None and not type(solution["choices"]) == float:
            question += f"Choices: {solution['choices']}\n"

    except:
        question = ""
        print("solution not found")
    new = "user: " + question + "\n" + "assistant: "
    for interaction in interactions:
        role = interaction["role"]
        if role == "assistant":
            new += interaction["content"]

    new += "\n\n"
    return new


good["chat"] = good.apply(create_Chat, axis=1)
good["length"] = good["chat"].apply(lambda x: len(word_tokenize(x)))
columns = good.columns

In [8]:
Base = """

Act as You are the professor of an important university Course.
The scenario is a answer explanation of a Teaching Assistant (assistant) of an exercise question of the course.
All TA responses are merged into a singular response.

You'll receive three categories of information enclosed within tags:
- QUESTION, detailing the exercise.
- SOLUTION, providing the answer to the exercise.
- ASSISTANT response, indicating the TA's reaction to the student's query.


Please follow these steps in the given order:
1. Analyze the QUESTION.
2. Assess the ASSISTANT response.
3. Review the SOLUTION.
4. Reformulate paraphrasing ASSISTANT explanation and the SOLUTION into a new, concise ASSISTANT response.

Your output should align with this format:
[Your Reformulation]

Ensure the output is coherent, maintains the essence of the assistant's response but paraphrasing it and the solution, and doesn't exceed 512 tokens.

Invest time in understanding each step and formulate your response thoughtfully.
"""

In [10]:
augmented = pd.DataFrame(columns=columns)
augmented

,confidence,interaction,interaction_id,sol_id,chat,assistant_chat,length


In [11]:
def paraphrase(example):
    Q_chat = Chat.create(f"Question chat 1")

    interactions = example["interaction"]
    last_role = interactions[0]["role"]
    last_content = interactions[0]["content"]

    new_interaction = []

    Solution = example["sol_id"]
    question = ""
    try:
        solution = solutions_df.loc[Solution]
        question += "<QUESTION>\n"
        question += f"Question: {solution['question']}\n"

        if solution["choices"] != None and not type(solution["choices"]) == float:
            question += f"Choices: {solution['choices']}\n"
        if (
            solution["explanation"] != None
            and not type(solution["explanation"]) == float
        ):
            question += f"Explanation: {solution['explanation']}\n"
        if (
            solution["description"] != None
            and not type(solution["description"]) == float
        ):
            question += f"Description: {solution['description']}\n"
        if solution["qtype"] != None and type(solution["qtype"]) != float:
            question += f"Type: {solution['qtype']}\n"

        question += "<QUESTION>\n\n"

        question += (
            "<SOLUTION>\n" + f"SOLUTION: {solution['answer']}\n" + "<SOLUTION>\n\n"
        )
    except:
        question = ""
        print("solution not found")

    instruction = ""
    if last_role == "system":
        instruction = last_content
        new_interaction.append({"role": last_role, "content": last_content})

    string = Base + question + "<ASSISTANT>" + "\n"
    try:
        chat = example["chat"].split("assistant: ")
        string += chat[1] + "\n" + "<ASSISTANT>\n"
        string += "\nREMEBER TO PARAPHRASE THE SOLUTION AND THE ASSISTANT RESPONSE"
        message = Q_chat.ask(content=string, instruction=instruction)
        message = message.content
        final_chat = chat[0] + message

        new_example = pd.DataFrame(columns=columns)
        new_example["confidence"] = [4]
        new_example["interaction_id"] = [Q_chat.chat_id]
        new_example["interaction"] = [new_interaction]
        new_example["chat"] = [final_chat]
        new_example["length"] = [len(word_tokenize(final_chat))]
        new_example["assistant_chat"] = [example["assistant_chat"]]
        new_example["sol_id"] = [example["sol_id"]]
        global augmented
        augmented = pd.concat([augmented, new_example], axis=0)
    except Exception as e:
        print(e)
        print("error")
        pass

    global counter
    counter += 1
    if counter % 20 == 0:
        print(time.time() - initial, "seconds")
        print(counter, "->", (counter / len(good)) * 1000 // 10, "%")
        print()


counter = 0
initial = time.time()

good.apply(paraphrase, axis=1)

53.476603984832764 seconds
20 -> 0.0 %

Server Error
error
121.55229377746582 seconds
40 -> 0.0 %

180.03838896751404 seconds
60 -> 1.0 %

229.1916699409485 seconds
80 -> 1.0 %

284.47879815101624 seconds
100 -> 2.0 %

335.20897698402405 seconds
120 -> 2.0 %

387.61037588119507 seconds
140 -> 2.0 %

445.3603730201721 seconds
160 -> 3.0 %

499.4931540489197 seconds
180 -> 3.0 %

556.1509461402893 seconds
200 -> 4.0 %

607.3647880554199 seconds
220 -> 4.0 %

654.3764679431915 seconds
240 -> 5.0 %

715.4000928401947 seconds
260 -> 5.0 %

767.5071589946747 seconds
280 -> 5.0 %

815.4421279430389 seconds
300 -> 6.0 %

862.0465421676636 seconds
320 -> 6.0 %

913.5376219749451 seconds
340 -> 7.0 %

963.2023169994354 seconds
360 -> 7.0 %

1021.8858330249786 seconds
380 -> 8.0 %

1073.372689962387 seconds
400 -> 8.0 %

1140.159017086029 seconds
420 -> 8.0 %

1200.7724521160126 seconds
440 -> 9.0 %

1249.1170690059662 seconds
460 -> 9.0 %

1304.198814868927 seconds
480 -> 10.0 %

1355.2199270725

5        None
6        None
7        None
10       None
13       None
         ... 
10196    None
10197    None
10207    None
10208    None
10209    None
Length: 4711, dtype: object

In [12]:
augmented.to_json("../data/paraphrase_one_assistant.json", orient="records", indent=4)

In [13]:
augmented[augmented["length"] <= 512]

,confidence,interaction,interaction_id,sol_id,chat,assistant_chat,length
0,4,[],863207,1001417,user: Assume that we have a convolutional neur...,1,280
0,4,"[{'role': 'system', 'content': 'You are a teac...",863210,1002278,user: Which of the following statements is \te...,1,286
0,4,"[{'role': 'system', 'content': 'You are an edu...",863214,1002278,user: Which of the following statements is \te...,1,150
0,4,"[{'role': 'system', 'content': 'Tu es un profe...",863216,1003546,user: D{\'e}terminez les lignes de courant $y ...,1,169
0,4,"[{'role': 'system', 'content': 'You are an edu...",863220,1004520,user: Assume [mathjaxinline] a_0=a_2 [/mathjax...,1,301
...,...,...,...,...,...,...,...
0,4,[],887804,2998144,user: Match one of the tools (a)-(e) to the fo...,1,150
0,4,"[{'role': 'system', 'content': 'You are trying...",887806,2998144,user: Match one of the tools (a)-(e) to the fo...,1,169
0,4,"[{'role': 'system', 'content': 'Could you plea...",887808,2999828,user: What does an ontology provide?\nChoices:...,1,137
0,4,"[{'role': 'system', 'content': 'From now on, y...",887811,2999828,user: What does an ontology provide?\nChoices:...,1,146


## Concat good interactions 
with the augmented, but just the ones that have one assistant_chat

In [ ]:
good["length"] = good["chat"].apply(lambda x: len(word_tokenize(x)))

In [255]:
good = good[good["assistant_chat"] == 1]

In [256]:
## concat augmented and good
concat = pd.concat([augmented, good], axis=0)
concat

,confidence,interaction,interaction_id,sol_id,chat,assistant_chat,length
0,4,"[{'role': 'system', 'content': 'You are a phys...",843682,1000851,user: Une conquille sphérique de rayon $R_1$ e...,3,339
0,4,"[{'role': 'system', 'content': 'QCM'}]",843694,1001199,user: Pour encoder la séquence de lettres ABCC...,2,266
0,4,[],843702,1001199,user: Pour encoder la séquence de lettres ABCC...,2,241
0,4,"[{'role': 'system', 'content': 'QCM'}]",843709,1001199,user: Pour encoder la séquence de lettres ABCC...,2,291
0,4,"[{'role': 'system', 'content': 'You are going ...",843717,1002413,user: Consider the Gaussian mixture model wher...,2,552
...,...,...,...,...,...,...,...
10196,5,"[{'role': 'user', 'content': ' Match one of th...",4384945,2998144,|USER| Match one of the tools (a)-(e) to the ...,1,NaN
10197,5,"[{'role': 'system', 'content': 'You are trying...",4994466,2998144,|SYS| You are trying to help students in quest...,1,NaN
10207,5,"[{'role': 'system', 'content': 'Could you plea...",4834259,2999828,|SYS| Could you please answer the multiple cho...,1,NaN
10208,4,"[{'role': 'system', 'content': 'From now on, y...",4823002,2999828,"|SYS| From now on, you will act as a computati...",1,NaN
